In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf

In [2]:
from alpha import iterate_minibatches 
from cnn_utils import conv_layer,fc_layer

In [3]:
import time, math
import matplotlib.pyplot as plt

import tqdm as tqdm
import os,cv2,random
import numpy as np
import pandas as pd
from collections import OrderedDict

%matplotlib inline

In [4]:
num_data = 99
IMG_SIZE = 64
labels = []
data = []
DIR = "./train"
for i, im in enumerate(os.listdir(DIR)):
  if not im.startswith('.'):
    path = os.path.join(DIR, im)
    if i >= num_data : break
    img = cv2.imread(path)
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    img = img/255.0
    data.append([np.array(img)])
    
    

In [5]:
train = np.array(data).reshape(-1,IMG_SIZE,IMG_SIZE,3)

In [6]:
train.shape

(99, 64, 64, 3)

In [7]:
np.save('./train.npy', train)

In [8]:
train = np.load('./train.npy').reshape(99, 64, 64, 3)

In [9]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17666194669463978261]

In [10]:
def saveGraph(graph):
  with tf.Session(graph = graph) as sess:
    filename = "./summary_log/VS-"+time.strftime("%H%M%S")
    writer = tf.summary.FileWriter(filename, sess.graph)
  print("Tensorboard summary saved to "+filename) 

In [11]:
graph1 = tf.Graph()

In [12]:
with graph1.as_default():
    X = tf.placeholder(tf.float32,[None,64,64,3])
    Y = tf.placeholder(tf.float32,[None,64,64,3])

In [13]:
def lrelu(b,alpha=0.1):
    return tf.maximum(alpha*b,b)

In [14]:
###Encoder

with graph1.as_default():
  with tf.name_scope('en-convolutions'):
    conv0 = tf.layers.conv2d(X, filters = 4, kernel_size  = (3, 3), strides = (1,1), padding = 'SAME', use_bias = True, activation = lrelu, name = 'conv0')
  # 32*32*4
  with tf.name_scope('en_pooling') :
    maxpool0 = tf.layers.max_pooling2d(conv0, pool_size = (2, 2), strides = (2, 2),name = 'pool0')
    
  with tf.name_scope('en-convolutions'):
    conv1 = tf.layers.conv2d(maxpool0, filters = 4, kernel_size  = (3, 3), strides = (1,1), padding = 'SAME', use_bias = True, activation = lrelu, name = 'conv1')
  
  # 32*32*4
  with tf.name_scope('en_pooling') :
    maxpool1 = tf.layers.max_pooling2d(conv1, pool_size = (2, 2), strides = (2, 2),name = 'pool0')
  # 16*16*4 
  with tf.name_scope('en-convolutions'):
    conv2 = tf.layers.conv2d(maxpool1, filters = 8, kernel_size  = (3, 3), strides = (1,1), padding = 'SAME', use_bias = True, activation = lrelu, name = 'conv2')
  # 16*16*8
  with tf.name_scope('en_pooling') :
    maxpool2 = tf.layers.max_pooling2d(conv2, pool_size = (2, 2), strides = (2, 2),name = 'pool0')
  # 8*8*8
  with tf.name_scope('en-convolutions'):
    conv3 = tf.layers.conv2d(maxpool2, filters = 16, kernel_size  = (3, 3), strides = (1,1), padding = 'SAME', use_bias = True, activation = lrelu, name = 'conv3')
  #  8x8x16
  with tf.name_scope('encoding'):
    encoded = tf.layers.average_pooling2d(conv3,pool_size=(2,2),strides=(2,2),name='encoding')
  

In [15]:
### decoder

with graph1.as_default():
  with tf.name_scope('decoder'):
    upsample1 = tf.layers.conv2d_transpose(encoded, filters = 16, kernel_size = 3, padding = 'SAME', strides = 2, name = 'upsample1')
    # 8*8*16
    conv4 = tf.layers.conv2d(upsample1, filters = 16, kernel_size = (3,3), strides = (1,1), padding = 'SAME', use_bias = True, name = 'conv4',activation =lrelu)
    # 8x8x16
    upsample2 = tf.layers.conv2d_transpose(conv4,filters=8,kernel_size=3,padding='same',strides=2,name='upsample2')
    # 16x16x8
    conv5 = tf.layers.conv2d(upsample2,filters=8,kernel_size=(3,3),strides=(1,1),name='conv5',padding='SAME',use_bias=True,activation=lrelu)
    # 16x16x8
    upsample3 = tf.layers.conv2d_transpose(conv5,filters=8,kernel_size=5,padding='same',strides=2,name='upsample3')
    # 32x32x8
    conv6 = tf.layers.conv2d(upsample3,filters=4,kernel_size=(5,5),strides=(1,1),name='conv6',padding='SAME',use_bias=True,activation=lrelu)
        
    # 16x16x8
    upsample4 = tf.layers.conv2d_transpose(conv6,filters=8,kernel_size=5,padding='same',strides=2,name='upsample4')
    # 32x32x8
    conv7 = tf.layers.conv2d(upsample4,filters=4,kernel_size=(5,5),strides=(1,1),name='conv7',padding='SAME',use_bias=True,activation=lrelu)
        
        
    # 32x32x4
    logits = tf.layers.conv2d(conv7,filters=3,kernel_size=(3,3),strides=(1,1),name='logits',padding='SAME',use_bias=True)
    #Now 32x32x3
    # Pass logits through sigmoid to get reconstructed image
    decoded = tf.sigmoid(logits,name='recon')

In [16]:
saveGraph(graph1)

Tensorboard summary saved to ./summary_log/VS-155013


In [17]:
with graph1.as_default():
  loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y)
  lr = tf.placeholder(tf.float32, shape=[])
  cost = tf.reduce_mean(loss) 
  opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost) 

In [19]:
minibatch_size = 256
epoch = 20
learning_rate = 0.0005


with tf.Session(graph=graph1) as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
#     saver.restore(sess, tf.train.latest_checkpoint('./saved_model/'))
    for ep in range(epoch):
      avg_cost = 0
      for i, minibatch in enumerate(iterate_minibatches(train, minibatch_size)):
        batch_cost, _ = sess.run([cost, opt], feed_dict={X: minibatch,
                                                             Y: minibatch,
                                                            lr: learning_rate})
        
      print("Epoch: {}/{}...".format(ep+1, epoch), \
                  "Training loss: {:.4f}".format(batch_cost))
    saver.save(sess, "./saved_model/model")

Epoch: 1/20... Training loss: 0.6936
Epoch: 2/20... Training loss: 0.6936
Epoch: 3/20... Training loss: 0.6934
Epoch: 4/20... Training loss: 0.6929
Epoch: 5/20... Training loss: 0.6925
Epoch: 6/20... Training loss: 0.6921
Epoch: 7/20... Training loss: 0.6917
Epoch: 8/20... Training loss: 0.6913
Epoch: 9/20... Training loss: 0.6910
Epoch: 10/20... Training loss: 0.6906
Epoch: 11/20... Training loss: 0.6902
Epoch: 12/20... Training loss: 0.6897
Epoch: 13/20... Training loss: 0.6892
Epoch: 14/20... Training loss: 0.6886
Epoch: 15/20... Training loss: 0.6880
Epoch: 16/20... Training loss: 0.6872
Epoch: 17/20... Training loss: 0.6863
Epoch: 18/20... Training loss: 0.6853
Epoch: 19/20... Training loss: 0.6842
Epoch: 20/20... Training loss: 0.6830
